In [30]:
# Imports

#estadistica y estructuras de datos
import pandas as pd
import numpy as np

#machine learning
import sklearn as sklearn
from sklearn.metrics import mean_squared_error

# normalizacion
from sklearn.preprocessing import StandardScaler

# regresion lineal
import sklearn.linear_model as lm

# division del conjunto en entrenamiento y testing
from sklearn.model_selection import train_test_split

# Cross Validation
from sklearn.model_selection import KFold

# en algun momento argumentamos a favor del coeficiente de determinacion como metrica valida
from sklearn.metrics import r2_score

# regularizacion
from sklearn.linear_model import Ridge, Lasso

# comparacion de precision
from sklearn.metrics import accuracy_score

# memory cache
from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose = 0)

# graficas
import seaborn as sn

# database connection
from ibmdbpy import IdaDataBase, IdaDataFrame

# errores y cosas para calcular z-scores
from numpy.linalg import pinv

# graficos
import matplotlib.pyplot as plt
%matplotlib inline

ImportError: No module named 'sklearn.model_selection'

In [17]:
# The code was removed by DSX for sharing.